In [1]:
import json
import requests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import urllib
from decouple import config

FDA_KEY = config('FDA_KEY')

In [2]:
url='https://api.nal.usda.gov/fdc/v1/foods/search?api_key=DEMO_KEY&query=Chocolate'
json_url = urllib.request.urlopen(url)
results = json.loads(json_url.read())

In [3]:
results
total_pages = results['totalPages']

In [4]:
flattened_data = pd.json_normalize(results)
flattened_data1 = pd.json_normalize(flattened_data.foods[0])
flattened_data1

,fdcId,description,lowercaseDescription,dataType,gtinUpc,publishedDate,brandOwner,ingredients,allHighlightFields,score,foodNutrients
0,730824,CHOCOLATE,chocolate,Branded,853508005332,2019-12-06,KAKAO CHOCOLATE,"DARK CHOCOLATE (CACAO BEANS, SUGAR, COCOA BUTT...",<b>Brand Owner</b>: KAKAO <em>CHOCOLATE</em><b...,478.29977,"[{'nutrientId': 1003, 'nutrientName': 'Protein..."
1,599449,CHOCOLATE,chocolate,Branded,747599408656,2019-04-01,Ghirardelli Chocolate Company,"SUGAR, COCOA BUTTER, WHOLE MILK POWDER, UNSWEE...",<b>Brand Owner</b>: Ghirardelli <em>Chocolate<...,477.50955,"[{'nutrientId': 1079, 'nutrientName': 'Fiber, ..."
2,358787,CHOCOLATE,chocolate,Branded,747599318351,2019-04-01,Ghirardelli Chocolate Company,"SUGAR, COCOA BUTTER, WHOLE MILK POWDER, SWEETE...",<b>Brand Owner</b>: Ghirardelli <em>Chocolate<...,477.50955,"[{'nutrientId': 1079, 'nutrientName': 'Fiber, ..."
3,533097,CHOCOLATE,chocolate,Branded,040232485503,2019-04-01,RENEE'S RAW CHOCOLATE,"ORGANIC RAW CACAO BUTTER, ORGANIC COCONUT, ORG...",<b>Brand Owner</b>: RENEE'S RAW <em>CHOCOLATE<...,477.50955,"[{'nutrientId': 1087, 'nutrientName': 'Calcium..."
4,358786,CHOCOLATE,chocolate,Branded,747599318368,2019-04-01,Ghirardelli Chocolate Company,"SUGAR, COCOA BUTTER, WHOLE MILK POWDER, UNSWEE...",<b>Brand Owner</b>: Ghirardelli <em>Chocolate<...,477.50955,"[{'nutrientId': 1087, 'nutrientName': 'Calcium..."
5,518457,CHOCOLATE,chocolate,Branded,747599325366,2019-04-01,Ghirardelli Chocolate Company,"SUGAR, UNSWEETENED CHOCOLATE, COCOA BUTTER, WH...",<b>Brand Owner</b>: Ghirardelli <em>Chocolate<...,477.50955,"[{'nutrientId': 1087, 'nutrientName': 'Calcium..."
6,452654,CHOCOLATE,chocolate,Branded,6543678520116,2019-04-01,FINE & RAW CHOCOLATE,"CACAO BEAN, COCONUT SUGAR, CACAO BUTTER, RASPB...",<b>Brand Owner</b>: FINE & RAW <em>CHOCOLATE</em>,477.50955,"[{'nutrientId': 1089, 'nutrientName': 'Iron, F..."
7,470630,CHOCOLATE,chocolate,Branded,012292530400,2019-04-01,NIAGARA CHOCOLATES A SWEETWORKS COMPANY,"MILK CHOCOLATE (SUGAR, COCOA BUTTER, MILK, CHO...",<b>Brand Owner</b>: NIAGARA <em>CHOCOLATES</em...,476.93857,"[{'nutrientId': 1079, 'nutrientName': 'Fiber, ..."
8,404560,CHOCOLATE,chocolate,Branded,093573921797,2019-04-01,RAW SHAKTI CHOCOLATE LLC,"ORGANIC RAW CACAO BEANS*, ORGANIC RAW COCONUT*...",<b>Brand Owner</b>: RAW SHAKTI <em>CHOCOLATE</...,476.91043,"[{'nutrientId': 1087, 'nutrientName': 'Calcium..."
9,852588,CHOCOLATE,chocolate,Branded,043319062307,2020-05-28,Supreme Chocolatier LLC,"MILK CHOCOLATE (SUGAR, COCOA BUTTER, MILK, CHO...",<b>Ingredients</b>: MILK <em>CHOCOLATE</em> (S...,476.85340,"[{'nutrientId': 1003, 'nutrientName': 'Protein..."


In [5]:
nutrients = flattened_data1['foodNutrients']
nutrients[0][0]

{'nutrientId': 1003,
 'nutrientName': 'Protein',
 'nutrientNumber': '203',
 'unitName': 'G',
 'derivationCode': 'LCCS',
 'derivationDescription': 'Calculated from value per serving size measure',
 'value': 5.71}

In [6]:
full_nutrient_list = [(nutrient_item['nutrientId'], nutrient_item['nutrientName']) for nutrient in nutrients for nutrient_item in nutrient]
nutrient_list = dict(set(full_nutrient_list)) #removes duplicates

In [ ]:
len(nutrient_list)

In [ ]:
nutrient_list

In [7]:
def get_sugar_details (sugar_id, nutrient_list):
    #returns dictionary containing sugar value
    for nutrient in nutrient_list:
        if nutrient['nutrientId'] == sugar_id:
            return nutrient['value']
    return np.nan


In [8]:
#extract nutrient_ids_related to sugar
sugar_nutrient_ids = dict(filter(lambda elem: 'sugar' in elem[1].lower(), nutrient_list.items()))
#add sugar columns
for sugar_id in sugar_nutrient_ids.items():
    flattened_data1[sugar_id[1]] = [get_sugar_details(sugar_id[0],nutrient_list) for nutrient_list in flattened_data1['foodNutrients']]




In [9]:
#rename sugar columns
flattened_data1 = flattened_data1.rename(columns = {'Sugars, total including NLEA':'Total_Sugars', 'Sugars, added':'Added_Sugars'})
#identify chocolate with the least amount of sugar
#(flattened_data1['Total_Sugars'].notnull().sum())
#(flattened_data1['Total_Sugars'].isna().sum())
sorted_chocolate = flattened_data1.sort_values(by=['Total_Sugars'])


In [10]:
#write to file
output_df = sorted_chocolate[['fdcId','brandOwner','ingredients','Total_Sugars','Added_Sugars']]
output_df.to_csv('chocolate.csv',index = False)

In [11]:
df=pd.DataFrame()

for i in range(1,total_pages):
    url=f'https://api.nal.usda.gov/fdc/v1/foods/search?api_key={FDA_KEY}&query=Chocolate&pageNumber={i}'
    json_url = urllib.request.urlopen(url)
    results = json.loads(json_url.read())
    flattened_data = pd.json_normalize(results)
    flattened_data = pd.json_normalize(results)
    flattened_data1 = pd.json_normalize(flattened_data.foods[0])
    df = df.append(flattened_data1)

HTTPError: HTTP Error 400: 

In [12]:
df

,fdcId,description,lowercaseDescription,dataType,gtinUpc,publishedDate,brandOwner,ingredients,allHighlightFields,score,foodNutrients,commonNames,additionalDescriptions,foodCode,ndbNumber
0,730824,CHOCOLATE,chocolate,Branded,853508005332,2019-12-06,KAKAO CHOCOLATE,"DARK CHOCOLATE (CACAO BEANS, SUGAR, COCOA BUTT...",<b>Brand Owner</b>: KAKAO <em>CHOCOLATE</em><b...,478.29977,"[{'nutrientId': 1003, 'nutrientName': 'Protein...",NaN,NaN,NaN,NaN
1,599449,CHOCOLATE,chocolate,Branded,747599408656,2019-04-01,Ghirardelli Chocolate Company,"SUGAR, COCOA BUTTER, WHOLE MILK POWDER, UNSWEE...",<b>Brand Owner</b>: Ghirardelli <em>Chocolate<...,477.50955,"[{'nutrientId': 1079, 'nutrientName': 'Fiber, ...",NaN,NaN,NaN,NaN
2,358787,CHOCOLATE,chocolate,Branded,747599318351,2019-04-01,Ghirardelli Chocolate Company,"SUGAR, COCOA BUTTER, WHOLE MILK POWDER, SWEETE...",<b>Brand Owner</b>: Ghirardelli <em>Chocolate<...,477.50955,"[{'nutrientId': 1079, 'nutrientName': 'Fiber, ...",NaN,NaN,NaN,NaN
3,533097,CHOCOLATE,chocolate,Branded,040232485503,2019-04-01,RENEE'S RAW CHOCOLATE,"ORGANIC RAW CACAO BUTTER, ORGANIC COCONUT, ORG...",<b>Brand Owner</b>: RENEE'S RAW <em>CHOCOLATE<...,477.50955,"[{'nutrientId': 1087, 'nutrientName': 'Calcium...",NaN,NaN,NaN,NaN
4,358786,CHOCOLATE,chocolate,Branded,747599318368,2019-04-01,Ghirardelli Chocolate Company,"SUGAR, COCOA BUTTER, WHOLE MILK POWDER, UNSWEE...",<b>Brand Owner</b>: Ghirardelli <em>Chocolate<...,477.50955,"[{'nutrientId': 1087, 'nutrientName': 'Calcium...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,628292,"CHOCOLATE CHIP BREAK 'N BAKE COOKIE DOUGH, CHO...","chocolate chip break 'n bake cookie dough, cho...",Branded,041303006757,2019-12-06,"Supervalu, Inc.","ENRICHED BLEACHED FLOUR [WHEAT FLOUR, NIACIN, ...","<b>Ingredients</b>: (VITAMIN B2), FOLIC ACID]...",-443.07214,"[{'nutrientId': 1003, 'nutrientName': 'Protein...",NaN,NaN,NaN,NaN
46,1016653,COCONUT ALMOND CHOCOLATE FINEST YOGHURT & CRUN...,coconut almond chocolate finest yoghurt & crun...,Branded,815909020161,2020-06-26,"Noosa Yoghurt, LLC",COCONUT YOGHURT (GRADE A PASTEURIZED WHOLE MIL...,"<b>Ingredients</b>: ], CANE SUGAR, WILDFLOWER ...",-443.07214,"[{'nutrientId': 1003, 'nutrientName': 'Protein...",NaN,NaN,NaN,NaN
47,1128157,"DARK AND WHITE CHOCOLATE CANDY CANE CARAMELS, ...","dark and white chocolate candy cane caramels, ...",Branded,797817202351,2020-11-13,Marich Confectionery Company,"WHITE CHOCOLATE (CANE SUGAR, COCOA BUTTER*, WH...",<b>Ingredients</b>: WHITE <em>CHOCOLATE</em> (...,-443.07214,"[{'nutrientId': 1003, 'nutrientName': 'Protein...",NaN,NaN,NaN,NaN
48,1115829,"OATMEAL CHOCOLATE CHIP THIN & CRISPY COOKIES, ...","oatmeal chocolate chip thin & crispy cookies, ...",Branded,044000050092,2020-11-13,Nabisco Biscuit Company,"SUGAR, UNBLEACHED ENRICHED FLOUR (WHEAT FLOUR,...","<b>Ingredients</b>: B1}, RIBOFLAVIN {VITAMIN ...",-443.07214,"[{'nutrientId': 1003, 'nutrientName': 'Protein...",NaN,NaN,NaN,NaN


In [13]:
for sugar_id in sugar_nutrient_ids.items():
    df[sugar_id[1]] = [get_sugar_details(sugar_id[0],nutrient_list) for nutrient_list in df['foodNutrients']]


In [14]:
#rename sugar columns
df = df.rename(columns = {'Sugars, total including NLEA':'Total_Sugars', 'Sugars, added':'Added_Sugars'})

#sort by Total Sugars descending
sorted_chocolate = df.sort_values(by=['Total_Sugars'])


In [15]:
#write to file
output_df = sorted_chocolate[['fdcId','brandOwner','ingredients','Total_Sugars','Added_Sugars']].head(1000)
output_df.to_csv('chocolate_1000k.csv',index = False)